In [ ]:
from __future__ import absolute_import
from __future__ import print_function
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import pandas as pd
import numpy as np
from keras.utils import np_utils 
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import losses
from keras.models import load_model
from keras import regularizers

In [ ]:
#Load the training dataset ~87K states
all_train = pd.read_csv("../input/applied-ai-assignment-2/Assignment_2_train.csv")
all_train.loc[(all_train.state == 4),'state']=0
all_train.loc[(all_train.state == 5),'state']=1

In [ ]:
len(all_train)

In [ ]:
all_train[1:5]

In [ ]:
#Create a train/validation split
data_to_use = .2
train=all_train[:int(len(all_train)*data_to_use)]
split = .9

Train = train[:int(len(train)*split)]
Valid = train[int(len(train)*split):]


#Remove the first and last column from the data, as it is the board name and the label
X_train = Train.iloc[:, 1:-1].values
X_valid = Valid.iloc[:, 1:-1].values

#Remove everything except the last column from the data, as it is the label and put it in y
y_train = Train.iloc[:, -1:].values
y_valid = Valid.iloc[:, -1:].values


In [ ]:
len(X_train)

In [ ]:
len(y_valid)

In [ ]:
sample_train = X_train[50].reshape(-1,6,7)[0]
sample_train

In [ ]:
import matplotlib.pyplot as plt
#plot the first image in the dataset
plt.imshow(sample_train)

In [ ]:
#set input to the shape of one X value
dimof_input = X_train.shape[1]

# Set y categorical
dimof_output = int(np.max(y_train)+1)
y_train = np_utils.to_categorical(y_train, dimof_output)
y_valid = np_utils.to_categorical(y_valid, dimof_output)


In [ ]:
y_valid

In [ ]:
# test the time

import time
from keras import initializers
glorot = initializers.glorot_normal(seed=None)

mlp_model = Sequential()

mlp_model.add(Dense(100, input_dim=dimof_input, kernel_initializer=glorot, activation='relu'))

mlp_model.add(Dense(dimof_output, kernel_initializer=glorot, activation='softmax')) #do not change

#suggested actvations
# 'relu'
# 'sigmoid'
# 'tanh'


#suggested optimizers 
sgd = optimizers.SGD(learning_rate=0.01) #default lr = 0.01
adagrad = optimizers.Adagrad(learning_rate=0.01) #default lr = 0.01
adadelta = optimizers.Adadelta(learning_rate=1.0) #default lr = 1.0
adam = optimizers.Adam(learning_rate=0.001) #default lr = 0.001
adamax = optimizers.Adamax(learning_rate=0.002) #default lr = 0.002
nadam = optimizers.Nadam(learning_rate=0.002) #default lr = 0.002


# Suggested loss functions
cat_cross = losses.categorical_crossentropy
mse = losses.mean_squared_error
binary = losses.binary_crossentropy


mlp_model.compile(loss=mse,   # **** pick any suggested loss functions
                  optimizer=adam, # **** pick any suggested optimizers
                  metrics=['accuracy']) #do not change

start = time.time()

history = mlp_model.fit(x=X_train, 
                        y=y_train, 
                        batch_size=10000, # **** set from 1 to length of training data
                        epochs=50, #pick any number higher will take longer
                        verbose=1, # allows you to see more info per epoch
                        validation_data=(X_valid,y_valid), 
                        validation_freq=1,
                        shuffle=True)

total_time = time.time()-start
print('Average time per epoch = '+str(total_time/5)+' seconds')
print('Aproximate time for 1000 epochs = '+str((total_time/5*1000)/60)+' minutes')

In [ ]:
mlp_model.summary()

In [ ]:
import time

mlp_model = Sequential()

mlp_model.add(Dense(100, input_dim=dimof_input, kernel_initializer=glorot, activation='relu'))


mlp_model.add(Dense(dimof_output, kernel_initializer=glorot, activation='softmax')) #do not change
#suggested actvations
# 'relu'
# 'sigmoid'
# 'tanh'


#suggested optimizers 
sgd = optimizers.SGD(learning_rate=0.01) #default lr = 0.01
adagrad = optimizers.Adagrad(learning_rate=0.01) #default lr = 0.01
adadelta = optimizers.Adadelta(learning_rate=1.0) #default lr = 1.0
adam = optimizers.Adam(learning_rate=0.001) #default lr = 0.001
adamax = optimizers.Adamax(learning_rate=0.002) #default lr = 0.002
nadam = optimizers.Nadam(learning_rate=0.002) #default lr = 0.002


# Suggested loss functions
cat_cross = losses.categorical_crossentropy
mse = losses.mean_squared_error
binary = losses.binary_crossentropy


mlp_model.compile(loss=mse,   # **** pick any suggested loss functions
                  optimizer=adam, # **** pick any suggested optimizers
                  metrics=['accuracy']) #do not change

start = time.time()

history = mlp_model.fit(x=X_train, 
                        y=y_train, 
                        batch_size=10000, # **** set from 1 to length of training data
                        epochs=50, #pick any number higher will take longer
                        verbose=0, # allows you to see more info per epoch
                        validation_data=(X_valid,y_valid), 
                        validation_freq=1,
                        shuffle=True)

# evaluate the model
_, train_acc = mlp_model.evaluate(X_train, y_train, verbose=0)
_, valid_acc = mlp_model.evaluate(X_valid, y_valid, verbose=0)
print('Train: %.3f, Valid: %.3f' % (train_acc, valid_acc))
print(time.time()-start)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
# model using early stopping and checkpoints
import time
from keras import initializers
glorot = initializers.glorot_normal(seed=None)

mlp_model = Sequential()

#create your network here

mlp_model.add(Dense(32, input_dim=dimof_input, kernel_initializer=glorot, activation='sigmoid'))
#mlp_model.add(BatchNormalization())
mlp_model.add(Dropout(0.5))
#mlp_model.add(BatchNormalization())

#mlp_model.add(Dense(64,kernel_initializer='uniform',kernel_regularizer=regularizers.l2(0.01), activation='relu'))


mlp_model.add(Dense(dimof_output, kernel_initializer=glorot, activation='softmax')) #do not change


#suggested optimizers 
sgd = optimizers.SGD(learning_rate=0.01) #default lr = 0.01
adagrad = optimizers.Adagrad(learning_rate=0.01) #default lr = 0.01
adadelta = optimizers.Adadelta(learning_rate=1.0) #default lr = 1.0
adam = optimizers.Adam(learning_rate=0.001) #default lr = 0.001
adamax = optimizers.Adamax(learning_rate=0.002) #default lr = 0.002
nadam = optimizers.Nadam(learning_rate=0.002) #default lr = 0.002


# Suggested loss functions
cat_cross = losses.categorical_crossentropy
mse = losses.mean_squared_error
binary = losses.binary_crossentropy


mlp_model.compile(loss=mse,   # **** pick any suggested loss functions
                  optimizer=adam, # **** pick any suggested optimizers
                  metrics=['accuracy']) #do not change

start = time.time()

es = EarlyStopping(monitor='val_loss', #do not change
                   mode='min',  #do not change
                   verbose=1, # allows you to see more info per epoch
                   patience=50) # **** patience is how many validations to wait with nothing learned (patience * validation_freq)

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True) #do not change

history = mlp_model.fit(x=X_train, 
                        y=y_train, 
                        batch_size=7900, # **** set from 1 to length of training data
                        epochs=10000, #do not change
                        verbose=0, # allows you to see more info per epoch
                        callbacks=[es, mc],
                        validation_data=(X_valid,y_valid), 
                        validation_freq=1,
                        shuffle=True)

#load the best model
saved_model = load_model('best_model.h5')

# evaluate the model
_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, valid_acc = saved_model.evaluate(X_valid, y_valid, verbose=0)
print('Train: %.3f, Valid: %.3f' % (train_acc, valid_acc))
print(time.time()-start)


In [ ]:
# Example grid search

import time
node = 2048*2
while node > 2:
    mlp_model = Sequential()

    mlp_model.add(Dense(node, input_dim=dimof_input, kernel_initializer='uniform', activation='relu'))
    mlp_model.add(Dropout(0.2))
    #mlp_model.add(BatchNormalization())
    #mlp_model.add(Dense(512,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.2))
    #mlp_model.add(Dense(64,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.5))
    #mlp_model.add(BatchNormalization())
    #mlp_model.add(Dense(32,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.3))
    #mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.3))

    mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax')) #do not change


    #suggested optimizers 
    sgd = optimizers.SGD(learning_rate=0.01) #default lr = 0.01
    adagrad = optimizers.Adagrad(learning_rate=0.01) #default lr = 0.01
    adadelta = optimizers.Adadelta(learning_rate=1.0) #default lr = 1.0
    adam = optimizers.Adam(learning_rate=0.001) #default lr = 0.001
    adamax = optimizers.Adamax(learning_rate=0.002) #default lr = 0.002
    nadam = optimizers.Nadam(learning_rate=0.002) #default lr = 0.002


    # Suggested loss functions
    cat_cross = losses.categorical_crossentropy
    mse = losses.mean_squared_error
    binary = losses.binary_crossentropy


    mlp_model.compile(loss=mse,   # **** pick any suggested loss functions
                      optimizer=adam, # **** pick any suggested optimizers
                      metrics=['accuracy']) #do not change

    start = time.time()

    es = EarlyStopping(monitor='val_loss', #do not change
                       mode='min',  #do not change
                       verbose=1, # allows you to see more info per epoch
                       patience=50) # **** patience is how many validations to wait with nothing learned (patience * validation_freq)

    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True) #do not change

    history = mlp_model.fit(x=X_train, 
                            y=y_train, 
                            batch_size=7900, # **** set from 1 to length of training data
                            epochs=10000, #do not change
                            verbose=0, # allows you to see more info per epoch
                            callbacks=[es, mc],
                            validation_data=(X_valid,y_valid), 
                            validation_freq=1,
                            shuffle=True)

    #load the best model
    saved_model = load_model('best_model.h5')

    # evaluate the model
    _, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
    _, valid_acc = saved_model.evaluate(X_valid, y_valid, verbose=0)
    print('Node - '+str(node))
    print('Train: %.3f, Valid: %.3f' % (train_acc, valid_acc))
    print(time.time()-start)
    node = int(node/2)

In [ ]:
# Test for standard deviation of a model

import time
valid_list = []
for i in range(10):
    mlp_model = Sequential()

    mlp_model.add(Dense(64, input_dim=dimof_input, kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.5))
    #mlp_model.add(BatchNormalization())
    #mlp_model.add(Dense(512,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.2))
    #mlp_model.add(Dense(64,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.5))
    #mlp_model.add(BatchNormalization())
    #mlp_model.add(Dense(32,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.3))
    #mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
    #mlp_model.add(Dropout(0.3))

    mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax')) #do not change


    #suggested optimizers 
    sgd = optimizers.SGD(learning_rate=0.01) #default lr = 0.01
    adagrad = optimizers.Adagrad(learning_rate=0.01) #default lr = 0.01
    adadelta = optimizers.Adadelta(learning_rate=1.0) #default lr = 1.0
    adam = optimizers.Adam(learning_rate=0.001) #default lr = 0.001
    adamax = optimizers.Adamax(learning_rate=0.002) #default lr = 0.002
    nadam = optimizers.Nadam(learning_rate=0.002) #default lr = 0.002


    # Suggested loss functions
    cat_cross = losses.categorical_crossentropy
    mse = losses.mean_squared_error
    binary = losses.binary_crossentropy


    mlp_model.compile(loss=mse,   # **** pick any suggested loss functions
                      optimizer=adam, # **** pick any suggested optimizers
                      metrics=['accuracy']) #do not change

    start = time.time()

    es = EarlyStopping(monitor='val_loss', #do not change
                       mode='min',  #do not change
                       verbose=1, # allows you to see more info per epoch
                       patience=50) # **** patience is how many validations to wait with nothing learned (patience * validation_freq)

    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True) #do not change

    history = mlp_model.fit(x=X_train, 
                            y=y_train, 
                            batch_size=7900, # **** set from 1 to length of training data
                            epochs=10000, #do not change
                            verbose=0, # allows you to see more info per epoch
                            callbacks=[es, mc],
                            validation_data=(X_valid,y_valid), 
                            validation_freq=1,
                            shuffle=True)

    #load the best model
    saved_model = load_model('best_model.h5')

    # evaluate the model
    _, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
    _, valid_acc = saved_model.evaluate(X_valid, y_valid, verbose=0)
    valid_list.append(valid_acc)
    print('Train: %.3f, Valid: %.3f' % (train_acc, valid_acc))
    print(time.time()-start)
print('Average Score- '+ str(np.mean(valid_list)))
print('Standard deviation - '+ str(np.std(valid_list)))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
#Go here when your saved model is ready
test = pd.read_csv("../input/applied-ai-assignment-2/Assignment_2_test.csv")
test.loc[(test.state == 4),'state']=0
test.loc[(test.state == 5),'state']=1
X_test = test.iloc[:, 1:-1].values
y_test = test.iloc[:, -1:].values


In [ ]:
#creates the final output 
list_of_boards = [i for i in list(test['file_names'])]
result = saved_model.predict(X_test)
test_results = []
for i in result:
    test_results.append(np.argmax(i))
#Creates a dataframe that can be saved as a csv for submission
submission_data = pd.DataFrame(
    {'BoardId': list_of_boards,
     'Label': test_results
    })

In [ ]:
submission_data[1:9]

In [ ]:
submission_data.to_csv('submission.csv', sep=',',index=False)